In [ ]:
import numpy as np
import array
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import eig 
import random
import xlwt
import itertools

In [ ]:
def noise(init_conf, num_iter, k): #Create a uniform distr. from 0 to 1. If the value is less than some k, write 0. otherwise, 1
    noise = np.zeros((init_conf.shape[0], num_iter))
    for j in range(0, num_iter):
        for i in range(0, init_conf.shape[0]):
            distr = np.random.uniform(0,1)
            if distr>k:
                noise[i,j] = 0
            else:
                noise[i,j] = 1
    return noise

In [ ]:
def rand_init_conf(s,m):
    init_conf = np.zeros(s)
    for i in range(s):
        distr = np.random.uniform(0,1)
        if distr>m:
            init_conf[i] = 0
        else:
            init_conf[i] = 1
            
    return init_conf

In [ ]:
def CA(rul, init_conf, num_iter, decomp):
    X = np.zeros((num_iter, init_conf.shape[0]))
    X[0,:] = init_conf
    Z = np.append(X,X, axis=1)
    X = np.append(Z,X, axis=1)
    for i in range(0, num_iter-1):
        for j in range(0, init_conf.shape[0]):
            for k in range(8):
                if np.array_equal(rul[k,:], X[i,(j+init_conf.shape[0]-1):(j+init_conf.shape[0]+2)]):
                    X[i+1,j+init_conf.shape[0]] = decomp[k]
                      
    Z = np.delete(X, slice(0, init_conf.shape[0]), 1)
    X = np.delete(Z, slice(init_conf.shape[0], Z.shape[1]), 1)
    return X

In [ ]:
def PCA(rul, init_conf, num_iter, noise, decomp):
    X = np.zeros((num_iter, init_conf.shape[0]))
    X[0,:] = init_conf
    for i in range(0, num_iter-1):
        for j in range(0, init_conf.shape[0]):
            if noise[j,num_iter-1-i] == 0:
                if j == 0:
                    for k in range(8):
                        if np.array_equal(rul[k,:],([X[i,init_conf.shape[0]-1],X[i,j],X[i,j+1]])):
                            X[i+1,j] = decomp[k]
                elif j+1<init_conf.shape[0]:
                    for k in range(8):
                        if np.array_equal(rul[k,:],([X[i,j-1],X[i,j],X[i,j+1]])):
                            X[i+1,j] = decomp[k]
                elif j+1==init_conf.shape[0]:
                    for k in range(8):
                        if np.array_equal(rul[k,:],([X[i,j-1],X[i,j],X[i,0]])):
                            X[i+1,j] = decomp[k]
            if noise[j,num_iter-1-i] == 1:
                if j == 0:
                    for k in range(8):
                        if np.array_equal(rul[k,:],([X[i,init_conf.shape[0]-1],X[i,j],X[i,j+1]])):
                            X[i+1,j] = 1-decomp[k]
                elif j+1<init_conf.shape[0]:
                    for k in range(8):
                        if np.array_equal(rul[k,:],([X[i,j-1],X[i,j],X[i,j+1]])):
                            X[i+1,j] = 1-decomp[k]
                elif j+1==init_conf.shape[0]:
                    for k in range(8):
                        if np.array_equal(rul[k,:],([X[i,j-1],X[i,j],X[i,0]])):
                            X[i+1,j] = 1-decomp[k]
    return X

In [ ]:
rule = 170
decomp = [int(x) for x in np.binary_repr(rule, width = 8)]
table = np.zeros((8,3))
for i in range(8):
    table[i,:] = [int(y) for y in np.binary_repr(7-i, width = 3)]

In [ ]:
s = 50
m = 0.5
init_conf = rand_init_conf(s,m)

In [ ]:
k = 1/s
num_iter = 100000 
noise_past = noise(init_conf, num_iter, k)#total noise

In [ ]:
nIterations_past = 10000 
noise_past_new = noise_past[:,0:nIterations_past]# how much we want to look into the past

In [ ]:
nIterations_past = 5000
noise_past_new = noise_past[:,0:nIterations_past] #how much we want to look into the past
q = 10 #number of slices
pullback = np.zeros((q,init_conf.shape[0]))
for i in range (0, q):
    pullback_single = PCA2(table, init_conf, nIterations_past-q+i, noise_past_new,decomp)
    pullback[i,:] = pullback_single[nIterations_past-q+i-1,:]


In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(pullback, interpolation='nearest', cmap='Greys')
plt.title('Pullback Object of Cellular Automata Rule {}'.format(rule))

In [ ]:
start = 10000
size = 50
end = 10500
q = 1
pullback = np.zeros((q*(end-start)//size+q,init_conf.shape[0]))
for j in range(start, end+size, size):
    noise_past_new = noise_past[:,0:j]
    for i in range (0, q):
        pullback_single = PCA(table, init_conf, j-q+i, noise_past_new,decomp)
        pullback[i+q*(j-start)//size,:] = pullback_single[j-q+i-1,:]

In [ ]:
    plt.figure(figsize = (10,10))
    plt.imshow(pullback, interpolation='nearest', cmap='Greys')
    plt.title('Pullback Object of Cellular Automata Rule {}'.format(rule))
    plt.yticks([])
    plt.savefig('Thesisplt/big_rule_{}.png'.format(rule),bbox_inches='tight')

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(pullback, interpolation='nearest', cmap='Greys')
plt.title('Pullback Object of Cellular Automata Rule {}'.format(rule))
#frame1 = plt.gca()
#frame1.axes.get_yaxis().set_visible(False)
plt.yticks([])
#plt.ylabel('5000 Iterations ', fontsize=12)
#plt.savefig('ElementaryCA/500_iterations_rule_{}.png'.format(rule))

In [ ]:
nIterations_past = 5000
noise_past_new = noise_past[:,0:nIterations_past]
q = 10

In [ ]:
for l in range (104,256): #several rules program
    rule = l
    decomp = [int(x) for x in np.binary_repr(rule, width = 8)]
    table = np.zeros((8,3))
    for i in range(8):
        table[i,:] = [int(y) for y in np.binary_repr(7-i, width = 3)]
    pullback = np.zeros((q,init_conf.shape[0]))
    for i in range (0, q):
        pullback_single = PCA(table, init_conf, nIterations_past-q+i, noise_past_new,decomp)
        pullback[i,:] = pullback_single[nIterations_past-q+i-1,:]
    plt.figure(figsize = (10,10))
    plt.imshow(pullback, interpolation='nearest', cmap='Greys')
    plt.title('Pullback Object of Cellular Automata Rule {}'.format(rule))
    plt.yticks([])
    #plt.savefig('Thesisplt/rule_{}.png'.format(rule),bbox_inches='tight')

In [ ]:
df = pd.DataFrame(noise_past)

In [ ]:
df.to_excel("noise_10_09.xlsx") #save noise